In [594]:
import pandas as pd
import json
from konlpy.tag import Okt
import numpy as np
import ast
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

In [550]:
train = pd.read_json('./origin_data_modify/train_mod5.json', typ='frame')
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 80953 entries, 0 to 80952
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   tags          80953 non-null  object
 1   id            80953 non-null  int64 
 2   plylst_title  80953 non-null  object
 3   songs         80953 non-null  object
 4   like_cnt      80953 non-null  int64 
 5   updt_date     80953 non-null  object
dtypes: int64(2), object(4)
memory usage: 4.3+ MB


- 태그 별 좋아요 수를 기준으로 sorting

In [551]:
train['tags'] = train['tags'].astype(str)
train['tags'][0][0]

'['

In [552]:
train[train['tags'] == "['']"]

,tags,id,plylst_title,songs,like_cnt,updt_date
5314,[''],58486,한국인듷,"[588648, 130871, 202564, 695590, 144582, 21015...",7,2017-10-11 09:19:27.000
20246,[''],26122,입문입문,"[580332, 10391, 223964, 144582, 256571, 232762...",5,2017-10-11 09:19:54.000
32338,[''],106238,아앙ㅇㅜㅠ,"[140638, 515089, 453750, 704374, 434027, 35001...",5,2017-10-11 09:16:58.000
51868,[''],52003,뚜두두둗ㄴ,"[57266, 332854, 99218, 44347, 7434, 520617, 27...",5,2017-10-11 09:17:55.000
51928,[''],74509,떨어진다아아,"[271384, 662434, 533893, 104230, 634553, 35038...",9,2017-10-11 09:18:17.000
69052,[''],118037,간지러워,"[434371, 288842, 184408, 662486, 233607, 55624...",5,2017-10-11 09:18:44.000


In [553]:
80953-6

80947

In [554]:
# tag 값을 갖고 있지 않는 플레이리스트 버리기
train = train[train['tags'] != "['']"]
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 80947 entries, 0 to 80952
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   tags          80947 non-null  object
 1   id            80947 non-null  int64 
 2   plylst_title  80947 non-null  object
 3   songs         80947 non-null  object
 4   like_cnt      80947 non-null  int64 
 5   updt_date     80947 non-null  object
dtypes: int64(2), object(4)
memory usage: 4.3+ MB


In [555]:
# 데이터프레임을 사용할 때, apply 함수를 사용하여 'tags' 열에 대해 변환을 적용
train['tags'] = train['tags'].apply(lambda x: ast.literal_eval(x))

In [556]:
train_copy = train.copy()
train_copy.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 80947 entries, 0 to 80952
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   tags          80947 non-null  object
 1   id            80947 non-null  int64 
 2   plylst_title  80947 non-null  object
 3   songs         80947 non-null  object
 4   like_cnt      80947 non-null  int64 
 5   updt_date     80947 non-null  object
dtypes: int64(2), object(4)
memory usage: 4.3+ MB


In [557]:
# 빈 태그 값을 갖고 있는 플레이리스트 추출

ply_id_del = [4435, 50403, 82487, 110957]
train_copy[train_copy['id'].isin(ply_id_del)]

,tags,id,plylst_title,songs,like_cnt,updt_date
54265,"[, 두근두근, 할수있을까, 과연, 올해는, 연애, 설레는]",4435,두근두근ㅇ_ㅇ 연애하고 싶게 만드는 달콤쏭,"[101572, 339005, 41567, 185929, 37965, 241866,...",7,2017-02-17 12:01:47.000
66170,"[힙합, Rap, 시즌2, 랩, HipHop, , 래퍼, 고등래퍼, 시즌1]",50403,고등래퍼 시즌1 참가자들의 방송 음원과 주요 발표곡들,"[658937, 204018, 616883, 399249, 27861, 6563, ...",47,2020-01-21 11:11:11.000
66804,"[겨울, 분위기, 매장음악, 카페, 연주곡, 잔잔한, 추억, 힐링, 일상, , 까페]",82487,겨울에 끝에서 듣는 힐링곡,"[106759, 151902, 655967, 138635, 393534, 67895...",4,2017-02-06 16:44:07.000
70884,"[, 가사로듣는, 영화]",110957,가사로 듣는 영화 17화 ㅋㅋ,"[612189, 219740, 569548, 471185, 188789, 19863...",2,2016-07-08 13:01:43.000


In [559]:
train_copy.at[54265, 'tags'] = ['두근두근', '할수있을까', '과연', '올해는', '연애', '설레는']
train_copy.at[66170, 'tags'] = ['힙합', 'Rap', '시즌2', '랩', 'HipHop', '래퍼', '고등래퍼', '시즌1']	
train_copy.at[66804, 'tags'] = 	['겨울', '분위기', '매장음악', '카페', '연주곡', '잔잔한', '추억', '힐링', '일상', '까페']
train_copy.at[70884, 'tags'] = ['가사로듣는', '영화']	

In [560]:
# 확인
train_copy[train_copy['id'].isin(ply_id_del)]

,tags,id,plylst_title,songs,like_cnt,updt_date
54265,"[두근두근, 할수있을까, 과연, 올해는, 연애, 설레는]",4435,두근두근ㅇ_ㅇ 연애하고 싶게 만드는 달콤쏭,"[101572, 339005, 41567, 185929, 37965, 241866,...",7,2017-02-17 12:01:47.000
66170,"[힙합, Rap, 시즌2, 랩, HipHop, 래퍼, 고등래퍼, 시즌1]",50403,고등래퍼 시즌1 참가자들의 방송 음원과 주요 발표곡들,"[658937, 204018, 616883, 399249, 27861, 6563, ...",47,2020-01-21 11:11:11.000
66804,"[겨울, 분위기, 매장음악, 카페, 연주곡, 잔잔한, 추억, 힐링, 일상, 까페]",82487,겨울에 끝에서 듣는 힐링곡,"[106759, 151902, 655967, 138635, 393534, 67895...",4,2017-02-06 16:44:07.000
70884,"[가사로듣는, 영화]",110957,가사로 듣는 영화 17화 ㅋㅋ,"[612189, 219740, 569548, 471185, 188789, 19863...",2,2016-07-08 13:01:43.000


In [600]:
train_copy.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 80947 entries, 0 to 80952
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   tags          80947 non-null  object
 1   id            80947 non-null  int64 
 2   plylst_title  80947 non-null  object
 3   songs         80947 non-null  object
 4   like_cnt      80947 non-null  int64 
 5   updt_date     80947 non-null  object
dtypes: int64(2), object(4)
memory usage: 6.3+ MB


In [601]:
train_copy.head()

,tags,id,plylst_title,songs,like_cnt,updt_date
0,[팝],50564,1 HITS OF DECADE SPECIAL 2,"[60025, 326802, 82991, 533287, 404318, 82686, ...",5,2009-07-11 21:37:42.000
1,"[기분좋은, 100번]",148495,100번 들어도 기분좋은 가요,"[208186, 155952, 669617, 438439, 397412, 60398...",4,2016-10-31 13:59:41.000
2,"[겨울, 크리스마스]",117162,12월의 재즈,"[588425, 407630, 235568, 347651, 194467, 42313...",353,2016-12-26 12:33:33.000
3,[발라드],151638,12월의 크리스마스를 yeah,"[290351, 404382, 591246, 18938, 610105, 676958...",29,2009-12-22 09:11:45.000
4,[일렉],27984,1월의 최신 EDM 소식,"[329736, 201677, 626680, 555898, 58608, 585855...",84,2015-01-29 16:22:19.000


In [615]:
train_copy_explode = train_copy.explode('tags').reset_index(drop=True)
train_copy_explode['tags'] = train_copy_explode['tags'].astype(str)
train_copy_explode.head()

,tags,id,plylst_title,songs,like_cnt,updt_date
0,팝,50564,1 HITS OF DECADE SPECIAL 2,"[60025, 326802, 82991, 533287, 404318, 82686, ...",5,2009-07-11 21:37:42.000
1,기분좋은,148495,100번 들어도 기분좋은 가요,"[208186, 155952, 669617, 438439, 397412, 60398...",4,2016-10-31 13:59:41.000
2,100번,148495,100번 들어도 기분좋은 가요,"[208186, 155952, 669617, 438439, 397412, 60398...",4,2016-10-31 13:59:41.000
3,겨울,117162,12월의 재즈,"[588425, 407630, 235568, 347651, 194467, 42313...",353,2016-12-26 12:33:33.000
4,크리스마스,117162,12월의 재즈,"[588425, 407630, 235568, 347651, 194467, 42313...",353,2016-12-26 12:33:33.000


In [621]:
tags_count = train_copy_explode.groupby('tags').count()['id'].reset_index()
tags_count.rename(columns={'id':'tags_count'}, inplace=True)
tags_count

,tags,tags_count
0,00,3
1,007,2
2,007시리즈,2
3,00s,1
4,00년,4
...,...,...
24442,힙해,5
24443,힙힙힙,2
24444,힛뎀포크,1
24445,힛뎀폭,1


In [626]:
tags_count['tags_count'].nunique()

342

In [623]:
tags_count.groupby('tags_count').count()['tags'].reset_index()

,tags_count,tags
0,1,14707
1,2,3283
2,3,1538
3,4,899
4,5,581
...,...,...
337,7142,1
338,7419,1
339,7907,1
340,8051,1


In [627]:
tags_count.describe()

,tags_count
count,24447.000000
mean,14.328098
std,191.896376
min,1.000000
25%,1.000000
50%,1.000000
75%,3.000000
max,11646.000000


In [612]:
from collections import Counter
 
pd.DataFrame(Counter(train_copy_explode['tags']).keys, Counter(train_copy_explode['tags']).values

<function Counter.keys>

In [ ]:
Counter(train_copy_explode['tags']).values

### KNN 도전

In [561]:
tag_like_cnt = train_copy.loc[:, ['tags', 'like_cnt', 'plylst_title', 'id']]
tag_like_cnt.head()

,tags,like_cnt,plylst_title,id
0,[팝],5,1 HITS OF DECADE SPECIAL 2,50564
1,"[기분좋은, 100번]",4,100번 들어도 기분좋은 가요,148495
2,"[겨울, 크리스마스]",353,12월의 재즈,117162
3,[발라드],29,12월의 크리스마스를 yeah,151638
4,[일렉],84,1월의 최신 EDM 소식,27984


In [562]:
tag_explode = tag_like_cnt.explode('tags').reset_index(drop=True)
tag_explode

,tags,like_cnt,plylst_title,id
0,팝,5,1 HITS OF DECADE SPECIAL 2,50564
1,기분좋은,4,100번 들어도 기분좋은 가요,148495
2,100번,4,100번 들어도 기분좋은 가요,148495
3,겨울,353,12월의 재즈,117162
4,크리스마스,353,12월의 재즈,117162
...,...,...,...,...
350274,비오는날,6,비오는날 듣기 좋은 연주곡 모음,106660
350275,추억,6,비오는날 듣기 좋은 연주곡 모음,106660
350276,회상,6,비오는날 듣기 좋은 연주곡 모음,106660
350277,비오는날,12,비 오는 날 어울리는 몽환적인 힐링 피아노 모음,150419


In [563]:
# 'tags' 열을 기준으로 그룹화하고 'like_cnt' 합하고 'plylst_title'을 새로운 리스트에 담음
tag_group = tag_explode.groupby('tags').agg({'like_cnt': 'sum', 'plylst_title': list, 'id': list}).reset_index()
tag_group

,tags,like_cnt,plylst_title,id
0,00,158,"[추억소환 신나는 댄스음악, Now and then 레트로 팝, I LOVE 20...","[38363, 81830, 123921]"
1,007,69,"[긴장감 넘치는 액션과 사운드 첩보 스파이물 대표 OST, 007 제임스 본드 테...","[89985, 45506]"
2,007시리즈,65,"[클래식한 스파이 액션의 대표주자 007 시리즈를 빛낸 역대 주제곡, 당대 최고 ...","[12845, 139999]"
3,00s,10,[00년대 많이 듣던 발라드 모음],[46831]
4,00년,30,"[미안함 사랑 그리움 다 느껴져 술 땡기는 밤듣기 좋은 발라드, 국내차트 올킬...","[53271, 68763, 67733, 112447]"
...,...,...,...,...
24442,힙해,2367,"[힙해 편집샵st 플레이리스트, 요즘 가장 핫한 2인조 프로듀싱팀 그루비룸의 음...","[40247, 119138, 152409, 76132, 71426]"
24443,힙힙힙,19,"[달달힙한노래 모음, 겨울에도 힙하게 얼죽힙 뉴트로한 뮤지션 모음집]","[59850, 45778]"
24444,힛뎀포크,41,[요즘 힙클 대세 Hit Them Folks를 아시나요],[37691]
24445,힛뎀폭,41,[요즘 힙클 대세 Hit Them Folks를 아시나요],[37691]


In [573]:
tag_group = tag_group.sort_values('like_cnt', ascending=False)
tag_group

,tags,like_cnt,plylst_title,id
5806,기분전환,1498902,"[2010 2017 빌보드 1위곡 모음, 5월의 이슈 & 힙합, 9월의 노래, AW...","[49998, 117722, 76457, 109204, 82403, 38561, 6..."
8086,드라이브,1242310,"[9월의 노래, AWESOME MIX VOL 1, HOT TRENDY 놓쳐선 안될 ...","[76457, 109204, 72795, 65164, 60630, 106060, 4..."
22142,팝,861909,"[1 HITS OF DECADE SPECIAL 2, All Number 1 Hits...","[50564, 109066, 138091, 151699, 20614, 123843,..."
24043,휴식,788773,"[9월의 노래, Daily, Deep Sleep, Good Night, My fav...","[76457, 13493, 42159, 14392, 34695, 106060, 74..."
4283,감성,769126,"[9월의 노래, Good Night, Happy song, K_RnB, MOOD N...","[76457, 14392, 31791, 29746, 127887, 98732, 34..."
...,...,...,...,...
19444,졸업장,2,[빛나는 졸업장과 수고한 나를 위한 선물 같은 음악],[24868]
633,8090락,2,[4 16 아침에 듣는 올드팝 올드락],[11277]
3316,jem,2,[내 고막 이미 jazz해 1 숨겨둔 내 심장에서 꺼내봤어 주섬주섬],[66210]
638,80년가요,2,[HS RADIO VOL 19 80년초반 나만의 인기 추억가요],[42531]


In [577]:
tag_group.head()

,tags,like_cnt,plylst_title,id,correct_tag
5806,기분전환,1498902,"[2010 2017 빌보드 1위곡 모음, 5월의 이슈 & 힙합, 9월의 노래, AW...","[49998, 117722, 76457, 109204, 82403, 38561, 6...",기분전환
8086,드라이브,1242310,"[9월의 노래, AWESOME MIX VOL 1, HOT TRENDY 놓쳐선 안될 ...","[76457, 109204, 72795, 65164, 60630, 106060, 4...",드라이브
22142,팝,861909,"[1 HITS OF DECADE SPECIAL 2, All Number 1 Hits...","[50564, 109066, 138091, 151699, 20614, 123843,...",팝
24043,휴식,788773,"[9월의 노래, Daily, Deep Sleep, Good Night, My fav...","[76457, 13493, 42159, 14392, 34695, 106060, 74...",휴식
4283,감성,769126,"[9월의 노래, Good Night, Happy song, K_RnB, MOOD N...","[76457, 14392, 31791, 29746, 127887, 98732, 34...",감성


In [597]:
okt = Okt()

tagged_data = [] # X
num = 0
for tag in tag_group['tags'] : 
  tagged_document = TaggedDocument(words=tag, tags=str(num))
  tagged_data.append(tagged_document)
  num += 1

len(tagged_data)

24447

In [599]:
tagged_data

[TaggedDocument(words='기분전환', tags='0'),
 TaggedDocument(words='드라이브', tags='1'),
 TaggedDocument(words='팝', tags='2'),
 TaggedDocument(words='휴식', tags='3'),
 TaggedDocument(words='감성', tags='4'),
 TaggedDocument(words='밤', tags='5'),
 TaggedDocument(words='잔잔한', tags='6'),
 TaggedDocument(words='새벽', tags='7'),
 TaggedDocument(words='카페', tags='8'),
 TaggedDocument(words='힐링', tags='9'),
 TaggedDocument(words='매장음악', tags='10'),
 TaggedDocument(words='신나는', tags='11'),
 TaggedDocument(words='Pop', tags='12'),
 TaggedDocument(words='스트레스', tags='13'),
 TaggedDocument(words='사랑', tags='14'),
 TaggedDocument(words='발라드', tags='15'),
 TaggedDocument(words='추억', tags='16'),
 TaggedDocument(words='힙합', tags='17'),
 TaggedDocument(words='여행', tags='18'),
 TaggedDocument(words='분위기', tags='19'),
 TaggedDocument(words='재즈', tags='20'),
 TaggedDocument(words='설렘', tags='21'),
 TaggedDocument(words='트렌디', tags='22'),
 TaggedDocument(words='알앤비', tags='23'),
 TaggedDocument(words='가을', tags='24'

In [ ]:
tag_group['correct_tag'] = tagged_data

In [598]:
# Doc2Vec 적용(문서 -> 숫자)
# dm=0 => DBOW 알고리즘 사용(단어 순서 고려 X)
# epochs => 같은 데이터에 대해 3번 학습을 반복
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

model = Doc2Vec(vector_size=20, dm=0, epochs=40)
model.build_vocab(tagged_data)

In [584]:
X = tagged_data
y = tag_group['correct_tag']
X.shape, y.shape

((24447, 3), (24447,))

In [590]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split

- input, target 변수 구분

In [586]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((19557, 3), (4890, 3), (19557,), (4890,))

In [588]:
knn = KNeighborsClassifier()

In [589]:
# 모델 훈련
knn.fit(X_train, y_train)

ValueError: could not convert string to float: '에너치'

In [ ]:
# 예측
y_pred = knn.predict(X_test)

ValueError: could not convert string to float: '벽난로'